In [2]:
import myutils
from myutils import load_corpus,stopwords


TRAIN_PATH = "C:/Users/11040/Desktop/weibonlp-master/movie/model/train.csv"
TEST_PATH = "C:/Users/11040/Desktop/weibonlp-master/movie/model/test.csv"

train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

import pandas as pd

df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_test = pd.DataFrame(test_data, columns=["words", "label"])

df_train.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11040\AppData\Local\Temp\jieba.cache
Loading model cost 0.780 seconds.
Prefix dict has been built successfully.


,words,label
0,原来 人死 后 还要 继续 干 生前 的 活儿 是 真的,1
1,刘德华 那条 线 有点儿 意思 为 这个 系列 电影 开启 了 新 维度 太长 了 删减 强...,1
2,不必 对比 什么 作为 里程碑 他 已经 无敌,1
3,这是 俄罗斯 的 型号 没有 安全 保险 只有 两个 扳机,1
4,年 看 第一部 的 时候 就 想着 回国 之后 如果 能 在 这样 的 剧组 工作 就 好 ...,1


In [3]:
# word2vec要求的输入格式: list(word)
wv_input = df_train['words'].map(lambda s: s.split(" "))  
wv_input.head()          

0             [原来, 人死, 后, 还要, 继续, 干, 生前, 的, 活儿, 是, 真的]
1    [刘德华, 那条, 线, 有点儿, 意思, 为, 这个, 系列, 电影, 开启, 了, 新,...
2                     [不必, 对比, 什么, 作为, 里程碑, 他, 已经, 无敌]
3             [这是, 俄罗斯, 的, 型号, 没有, 安全, 保险, 只有, 两个, 扳机]
4    [年, 看, 第一部, 的, 时候, 就, 想着, 回国, 之后, 如果, 能, 在, 这样...
Name: words, dtype: object

In [4]:
from gensim import models

In [5]:
word2vec = models.Word2Vec(wv_input,
                          vector_size = 64,
                          min_count = 1,
                          epochs =1000)

In [54]:
word2vec.wv.most_similar("电影")

[('科幻', 0.6783728003501892),
 ('科幻片', 0.6240063905715942),
 ('了', 0.5985647439956665),
 ('的', 0.5743614435195923),
 ('是', 0.5600840449333191),
 ('科幻电影', 0.5403375625610352),
 ('片子', 0.5037230849266052),
 ('也', 0.49708935618400574),
 ('我', 0.4955514073371887),
 ('作品', 0.4937562644481659)]

In [55]:
word2vec.wv.most_similar("伤心")

[('抓狂', 0.9487404227256775),
 ('tanshou', 0.9330409169197083),
 ('jiayou', 0.912369966506958),
 ('haha', 0.9112916588783264),
 ('heiheihei', 0.9071836471557617),
 ('非杠', 0.9063233137130737),
 ('没说', 0.9022008776664734),
 ('xiaoku', 0.8585681915283203),
 ('bingbujiandan', 0.8550329208374023),
 ('shangxin', 0.8495885729789734)]

In [22]:
pip install torch

     -------------------------------------- 162.6/162.6 MB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from torch.utils.data import Dataset, DataLoader

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# 超参数
learning_rate = 5e-4
input_size = 768
num_epoches = 5
batch_size = 100
embed_size = 64
hidden_size = 64
num_layers = 2

In [7]:
import numpy as np
# 数据集
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.label2 = df["label"].tolist()
        self.label = np.array(self.label2,dtype=int) 
        for s in df["words"].tolist():
            vectors = []
            for w in s.split(" "):
                if w in word2vec.wv.key_to_index:
                    vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
            vectors = torch.Tensor(vectors)
            self.data.append(vectors)
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    """
    :param data: 第0维：data，第1维：label
    :return: 序列化的data、记录实际长度的序列、以及label列表
    """
    data.sort(key=lambda x: len(x[0]), reverse=True) # pack_padded_sequence要求要按照序列的长度倒序排列
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # 用RNN处理变长序列的必要操作
    return data, torch.tensor(y, dtype=torch.float32), data_length


# 训练集
train_data = MyDataset(df_train)
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# 测试集
test_data = MyDataset(df_test)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

C:\Users\11040\AppData\Local\Temp\ipykernel_15352\253272313.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  vectors = torch.Tensor(vectors)


In [8]:
# 网络结构
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # 双向, 输出维度要*2
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # 双向, 第一个维度要*2
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)  # 双向, 所以要将最后两维拼接, 得到的就是最后一个time step的输出
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        return out

lstm = LSTM(embed_size, hidden_size, num_layers)

In [9]:
from sklearn import metrics

# 在测试集效果检验
def test():
    y_pred, y_true = [], []

    with torch.no_grad():
        for x, labels, lengths in test_loader:
            x = x.to(device)
            outputs = lstm(x, lengths)          # 前向传播
            outputs = outputs.view(-1)          # 将输出展平
            y_pred.append(outputs)
            y_true.append(labels)

    y_prob = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    y_pred = y_prob.clone()
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred <= 0.5] = 0
    
    print(metrics.classification_report(y_true, y_pred))
    print("准确率:", metrics.accuracy_score(y_true, y_pred))
    print("AUC:", metrics.roc_auc_score(y_true, y_prob) )

# 定义损失函数和优化器
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# 迭代训练
for epoch in range(num_epoches):
    total_loss = 0
    for i, (x, labels, lengths) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        outputs = lstm(x, lengths)          # 前向传播
        logits = outputs.view(-1)           # 将输出展平
        loss = criterion(logits, labels)    # loss计算
        total_loss += loss
        optimizer.zero_grad()               # 梯度清零
        loss.backward(retain_graph=True)    # 反向传播，计算梯度
        optimizer.step()                    # 梯度更新
        if (i+1) % 10 == 0:
            print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
            total_loss = 0
    
    # 测试集效果检验
    test()
    
    model_path = "C:/Users/11040/Desktop/weibonlp-master/sentiments/model/lstm_{}.model".format(epoch+1)
    torch.save(lstm, model_path)
    print("saved model: ", model_path)

epoch:1, step:10, loss:0.6848745346069336
epoch:1, step:20, loss:0.6560856103897095
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        69
         1.0       0.65      0.99      0.79       131

    accuracy                           0.65       200
   macro avg       0.33      0.50      0.39       200
weighted avg       0.43      0.65      0.52       200

准确率: 0.65
AUC: 0.6552162849872774
saved model:  C:/Users/11040/Desktop/weibonlp-master/sentiments/model/lstm_1.model
epoch:2, step:10, loss:0.6238754987716675
epoch:2, step:20, loss:0.6135467886924744
              precision    recall  f1-score   support

         0.0       1.00      0.03      0.06        69
         1.0       0.66      1.00      0.80       131

    accuracy                           0.67       200
   macro avg       0.83      0.51      0.43       200
weighted avg       0.78      0.67      0.54       200

准确率: 0.665
AUC: 0.7511339749972342
saved model:  C:/Users/110

In [49]:
pip install --upgrade numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
net = torch.load("C:/Users/11040/Desktop/weibonlp-master/sentiments/model/lstm_5.model") 

from myutils import processing

strs = ["看吐了 难看爆了", "真牛逼！中国电影工业！"]

data = []
for s in strs:
    vectors = []
    for w in processing(s).split(" "):
        if w in word2vec.wv.key_to_index:
            vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
    vectors = torch.Tensor(vectors)
    data.append(vectors)
x, _, lengths = collate_fn(list(zip(data, [-1] * len(strs))))
with torch.no_grad():
    x = x.to(device)
    outputs = lstm(x, lengths)          # 前向传播
    outputs = outputs.view(-1)          # 将输出展平
print(outputs)

tensor([0.2515, 0.9509])
